In [1]:
import os

from pydantic import BaseModel, Field
from IPython.display import Image, display
from typing import Annotated, TypedDict, List

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [2]:
from dotenv import load_dotenv

load_dotenv("/home/ubuntu/LLM-tutorials/keys.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]

In [4]:
########## 2. 도구 정의 및 바인딩 ##########
search_tool = TavilySearchResults(k=5)
tools = [search_tool]

In [5]:
llm = ChatOpenAI(model_name="gpt-4.1-nano")
llm_with_tools = llm.bind_tools(tools)

In [6]:
########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [7]:
# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[search_tool]) ## callable이 전달되어야 함. tools 리스트를 전달해서는 안된다.

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)


In [8]:
########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

In [9]:
# 그래프 빌더 컴파일
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)


In [10]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10, ## 최대 10개의 노드를 방문. 그 이상은 RecursionError 발생
    configurable = {"thread_id" : "1"}
)

In [11]:
# 첫 질문
question = ("내 이름은 'kevin de bryne' 입니다. 만나서 반가워요")

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

반갑습니다, Kevin De Bryne님! 만나서 정말 기쁩니다. 어떤 도와드릴 일이 있으신가요?


In [12]:
# 이어지는 질문
question = "내 이름이 뭐라고 했지?"

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

당신의 이름은 "Kevin De Bryne"라고 하셨습니다.


In [ ]:
## configurable에서 thread_id가 변경되면 새로운 스레드가 생성되기 때문에 이전 스레드에서 했던 대화 내용을 기억하지 못한다.

question = "내 이름이 뭐라고 했지?"

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "2"},  # 스레드 ID 설정
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()
